In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import scipy.optimize as sciopt
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates

import matplotlib.style as mplstyle
mplstyle.use('fast')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import multiprocessing as mp

In [ ]:
#the data will be located in the directory "data_dir", with filenames in the list "data_files". File extension may need to be changes, this code was written to run in a Linux environment
import os
data_dir = 'data/'
data_files = ["BH016-1_2GyroidPDMS-Dev_F200_coated_08_04_2024_Stability_in _gasToluene_Freq_stab_120sccm3X_5VPowerBank.txt",
              "SS123-3_1-Dev_F200_coated_08_02_2024_Stability_in_gas_Toluene_Freq_stab_120sccm3X_5VPowerbank.txt",
              "SS066-4_2-Gyroid75PUT-[Benchtop]Dev_F200OX_coated_10_26_2024_Stability_in _120sccm1XgasToluene_5VPowersupply2.txt"]
              # "SS066-4_2-Gyroid75PUT-[Benchtop]Dev_F200OX_coated_10_28_2024_Stability_in _120sccmVsBlockPUT1XgasToluene_5VPowersupply.txt"]
              # "SS066-4_2-Gyroid75PUT-[Benchtop]Dev_F200OX_coated_10_26_2024_Stability_in _120sccm1XgasToluene_5VPowersupply2.txt",
            #   "SS123-3_1-Dev_F200_coated_03_12_2025_in_gasToluene_Freq_stab_120sccm10min1X_5VPowerSupply.txt",

nice_names = ["PDMs Gyroid",
              "Spraycoated", #I think it's PDMS, but I need confirmation
              "PUT Gyroid"]
print(f'Your OS sees you\'re current working directory location as \"{os.getcwd()}\" \n from that location, files: \n-  \"{'\"\n-  \"'.join(data_files)}\"\nshould be found in directory \"{data_dir}\" (with indices {[r for r in range(len(data_files))]} respectively)')

In [ ]:
## load data here
#read all dataset files, and create a list of pandas dataframes, indexed by time relative to epoch time (for simplicity)
datasets = []
for file in data_files:
    df = pd.read_csv(data_dir+file,sep ='\t',engine='python',header=1,skipfooter=1)
    df['datetime'] = pd.to_datetime(df['Time [ms]'],unit='ms')
    df = df.set_index('datetime')
    df['load'] = [[]] * len(df)
    df['MFC [ml/min]'] = [[]] * len(df)
    df['cycle_id'] = [[""]] * len(df)
    datasets.append(df)


# print(df)

In [ ]:
def addloads(df,offset,period,load_cycle):
    
    y = df.resample(period,label='left',offset=offset)
    # print(df.index.to_period('5m'))

    df['load'] = [[]] * len(df)
    df['MFC [ml/min]'] = [[]] * len(df)
    df['cycle_id'] = [[""]] * len(df)

    for i,sample in enumerate(y):
        # from the sample in y, get just the groups of points associated with the sample (the first index is just a timestamp)
        group = sample[1]
        #this command is a bit weird, it will calculate which index to pass to 'load_cycle' based on index 'i'. The interior most divmod() accounts for the 2 parts of each cycle (load and purge), the outer most accounts for the number of different loads encountered in one period of tests
        cycle_divmod = divmod(divmod(i-1,2)[0],10)
        
        #assign the flow rate for the sample, making note that this assigns a nonzero flow rate to the purge cycle as well. I believe the purge data could have some use down the line, and would be most useful if the purge's history is known (i.e. what concentration it is pumping down from)
        df.loc[group.index, 'MFC [ml/min]'] = [load_cycle[cycle_divmod[1]]] *  len(group)
        #Toggle df field assignment based on whether purging or loading
        if divmod(i+1,2)[1]==1: #purge
            df.loc[group.index,'load'] = [False] * len(group) #False means no loading (purge)
            #add unique descriptor to 'cycle_id'; this could be optimized if speed is an issue. For now, it's best to keep it human readable
            df.loc[group.index,'cycle_id'] = [f'{'rising' if np.sign(np.real((-1)**(-(i-5)/10))) == 1 else 'falling'} cycle {divmod(divmod(i,2)[0],10)[0]+1}; purge from {load_cycle[cycle_divmod[1]]} ml/min.'] * len(group)
        else: #load
            df.loc[group.index,'load'] = [True] * len(group) #True means loading
            #add unique descriptor to 'cycle_id'; this could be optimized if speed is an issue. For now, it's best to keep it human readable
            df.loc[group.index,'cycle_id'] = [f'{'rising' if np.sign(np.real((-1)**(-(i-5)/10))) == 1 else 'falling'} cycle {divmod(divmod(i,2)[0],10)[0]+1}; {load_cycle[cycle_divmod[1]]} ml/min. load'] * len(group)


In [ ]:
## Clean up data here
# initialize ramp rates
load_ramp = [10,20,30,40,50] 
load_cycle = load_ramp+load_ramp[::-1] #the ramp down is just the mirror of the ramp up

#set data offsets, and cycle periods. This will take some trial and error to get right, but plotly makes it easy to zoom and interact with your data
offsets = ['1.50106min','3.47775min','3.16415min',f'{1.29528}min'] #'1.34286min'
periods = ['5.001475927min'] * len(datasets) #same period used for all of the data
load_cycles = [load_cycle] * len(datasets)
# print()
#don't worry about an output, the function applies updates directly to the dataframes you're feeding it from the `datasets` list
_ = [addloads(df,offset,period,load_cycle) for (df,offset,period,load_cycle) in zip(datasets,offsets,periods,load_cycles)]


#visualize in Plotly in one big plot
fig = make_subplots(rows=1,cols=1)

#for each dataframe (whole data now)
for i,df in enumerate(datasets):
    #filter by only the values in the dataframe where there is active loading, and is in 'cycle 1'
    df_view = df.where(df['load']).where((df['cycle_id'].str.contains('cycle 1'))) # | df['cycle_id'].str.contains('cycle 2') #add this inside the parethesis of the rightmost .where() function if you want to look at the second cycle too
    
    #add the scattter plot to the plotly figure
    fig.append_trace(go.Scatter(x = df_view.index - pd.to_timedelta(df_view['Time [ms]'].min(),unit='milliseconds'), #x = df_view['Time [ms]'].subtract(df_view['Time [ms]'].min()),
                                y = df_view['Frequency [Hz]'].sub(df_view.loc[df['load']].iloc[0,1]),name=data_files[i][:7]),
                                row=1,
                                col=1)

#export option is great if you want to pull up the plot in the browser, if you double-click the file in file explorer, it's likely your OS will automatically open it in the default browser app.
fig.write_html('allDatapoints.html')
fig.show()

In [ ]:
import import_ipynb
from notebooks import diffusionModel as diffmod

In [ ]:

def residual_eqn1(x,A,h,t,dm):#(D,A,h,t,f):
    D = x[0]*1e-14
    dt = x[1]
    if dt<=0:
        t_plus_dt = t+dt
        idx = np.where(t_plus_dt>=0)[0]
        t = t_plus_dt[idx]
        dm = dm[idx]
    attempt = diffmod.eqn1(D,A,h,t)
    diff = np.diff(np.vstack((dm,attempt)),axis=0).squeeze()
    rms_diff = np.sqrt(np.mean(np.square(diff)))
    # print(diff.shape,dm.shape,attempt.shape)
    # print(rms_diff,D*1e14)
    return rms_diff

# def residual_eqn3(D,A,h,t,dm):#(D,A,h,t,f):
#     D = D*1e-14
    
#     attempt = diffmod.eqn3(D,A,h,t)
#     diff = np.diff(np.vstack((dm,attempt)),axis=0).squeeze()
#     rms_diff = np.sqrt(np.mean(np.square(diff)))
#     # print(diff.shape,dm.shape,attempt.shape)
#     # print(rms_diff,D*1e14)
#     return rms_diff


In [ ]:
## Fit data to diffusion equations here
D = 3e-8/(100**2) #m^/s diffusion coeff
#arguments
h = 5e-6 #meters, actual height of a hammerhead is ~5 um

fig = make_subplots(rows=1,cols=1)
output_fits = []

for i,df in enumerate(datasets):
    #filter dataframe by those in the VOC "load" phase for cycle 1. If multiple cycles are needed, add an OR operator ("|") and another "(df['cycle_id'].str.contains('cycle #'))"
    df_filtered = df.loc[(df['load']) & 
                         (df['cycle_id'].str.contains('cycle 1'))]# &
                        #  (df['MFC [ml/min]']==50)]
    
    #baseline frequency, the "zero" for all data. Taken at the start of the first load curve
    f0 = df.loc[df['load']].iloc[0,1]

    #concatenate unique names for all cycles in specified range
    cycle_id = df_filtered['cycle_id'].unique()

    for cid in cycle_id: #for each cycles we're interested in
        #look at each curve individually
        set_df = df_filtered.loc[(df['cycle_id']==cid)]
        df_max = set_df['Frequency [Hz]'].sub(f0).min() #minimum bc it's in the negative direction
        dm_max = -2*df_max
        
        #threshold for the two fits
        set_0p5 = set_df.loc[set_df['Frequency [Hz]'].sub(f0) > 0.5*df_max]
        set_0p8 = set_df.loc[set_df['Frequency [Hz]'].sub(f0) > 0.8*df_max]
        
        #convert to numpy arrays, with appropriate dm and t offsets
        set_0p5_dm = set_0p5['Frequency [Hz]'].sub(f0).to_numpy()*-2
        set_0p8_dm = set_0p8['Frequency [Hz]'].sub(f0).to_numpy()*-2
        
        set_0p5_t = set_0p5['Time [ms]'] - set_0p5['Time [ms]'].min()
        set_0p8_t = set_0p8['Time [ms]'] - set_0p8['Time [ms]'].min()
        set_0p5_t = set_0p5_t.to_numpy()/1000
        set_0p8_t = set_0p8_t.to_numpy()/1000

        #do fit for eqn 1; this part is heavy and would be more performant if parallelized, but the data handoff would take more code
        res1 = sciopt.minimize(residual_eqn1,
                                [D,-0.5],
                                bounds=[(0.1,10000),(-1,10)],
                                args=(dm_max,h,set_0p8_t,set_0p8_dm))
        D_eff1 = res1.x[0]*1e-14
        dt_eff1 = res1.x[1]*1e-14
        r2_eqn1 = r2_score(set_0p8_dm,diffmod.eqn1(D_eff1,dm_max,h,set_0p8_t))

        #do fit for eqn 3
        p = np.polyfit(np.sqrt(set_0p5_t),set_0p5_dm,1)
        r2_eqn3 = r2_score(set_0p5_dm,np.polyval(p,np.sqrt(set_0p5_t)))
        D_eff3 = ((p[0]*h/(4*np.abs(df_max)))**2)*np.pi #the /1000 factor is to account for the time being in milliseconds
        dt_eff3 = p[1]/p[0] #find the y-intercept; which is going to be an effective "dt" offset from what would be considered a real t0 for an ideal system

        output_fits.append({'filename': data_files[i],
                            'cycle_id': cid,
                            'eqn1 Deff1 (cm^2/s)': D_eff1*(100**2),
                            'eqn1 R^2': r2_eqn1,
                            'eqn1 dt': dt_eff1,
                            'eqn3 Deff3 (cm^2/s)': D_eff3*(100**2),
                            'eqn3 R^2': r2_eqn3,
                            'eqn3 dt': dt_eff3,
                            'MFC [ml/min]': set_df['MFC [ml/min]'].unique()[0],
                            'load': set_df['load'].unique()[0],
                            'sampling period (min)': periods[i][:-3],
                            'sampling offset (min)': offsets[i][:-3]})
        # print(f'Eqn. 1; D = {D_eff1*(100**2)*(10**10)}x10^-10 cm^2/s; R^2 = {r2_eqn1}; dt = {dt_eff1}')
        # print(f'Eqn. 3; D = {D_eff3*(100**2)*(10**10)}x10^-10 cm^2/s; R^2 = {r2_eqn3}; dt = {dt_eff3}')

        # fig.append_trace(go.Scatter(x = set_df['Time [ms]'].div(1000) - set_df['Time [ms]'].min()/1000,
        #                             y = -2*set_df['Frequency [Hz]'].sub(f0), name=data_files[i][:7]),
        #                             row=1,
        #                             col=1)
        
        # fig.append_trace(go.Scatter(x = set_0p5_t/1000-dt_eff3,
        #                             y = diffmod.eqn3(Deff3,-2*df_max,h,set_0p5_t/1000-dt_eff3)+p[1],
        #                             name='fit',
        #                             # fill = 'black',
        #                             line=dict(dash='dot',color='black')),
        #                             row=1,
        #                             col=1)
# fig.show()


In [ ]:
import csv
from datetime import datetime
# print(output_fits)

output_file = data_dir + f'output_diffusion_fits_{datetime.now().strftime('%Y-%m-%d_%H:%M')}.csv'
fieldnames = ['filename','cycle_id','eqn1 Deff1 (cm^2/s)','eqn1 R^2','eqn1 dt','eqn3 Deff3 (cm^2/s)','eqn3 R^2','eqn3 dt','MFC [ml/min]','load','sampling period (min)','sampling offset (min)']
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output_fits)

In [ ]:
##scratch
# df = datasets[0]
# print(np.sum(df['cycle_id'].str.contains('load').to_numpy()))
df = datasets[0]
# print(len(df.where(df['load'])))
# print(df.loc[df['cycle_id'].str.contains('cycle 1')]['cycle_id'].unique())
# print(len(df.loc[df['load'] & df['cycle_id'].str.contains('cycle 1')]))
# print(datasets[0].index - pd.to_timedelta(datasets[0]['Time [ms]'].min(),unit='milliseconds'))
# print(df['Frequency [Hz]'].sub(df.iloc[0,1]-1))
# print([df['cycle_id'].uni?que() for df in datasets])
# df.where(df['cycle_id'].str.contains('purge'))['Frequency [Hz]']
# figure = px.scatter(df.where(df['cycle_id'].str.contains('purge'))['Frequency [Hz]'])

# #scratch for data labelling
# print(''.join([f'{'rising' if np.sign(np.real((-1)**(-(i-6)/10))) == 1 else 'falling'} cycle {divmod(divmod(i-1,2)[0],10)[0]+1}; purge from {load_cycle[divmod(divmod(i-2,2)[0],10)[1]]} ml/min.\n' for i in range(25)]))


In [ ]:
#plots everything you imported
fig = make_subplots(rows=1,cols=1)

for filename,df in zip(data_files,datasets):    
    fig.append_trace(go.Scatter(x = df.index, y = df['Frequency [Hz]'].subtract(df.loc[df.index[0],'Frequency [Hz]']),),row=1,col=1)

# fig.write_html('allDatapoints.html')
fig.show()

In [ ]:
# print(df.loc[df.index[0],'Frequency [Hz]'])